In [ ]:
# artifact_id_dict = {
#     # # run-{}-train_metric_table:v0  
#     # "flowbot": 'run-pjnc1g2v-simulation_metric_table:v827',  # without mask
#     # # "traj1": 'wcx2tyyn',
#     # "traj5": 'run-zes1lcl9-simulation_metric_table:v829',
#     # "traj10": 'run-bhto6if3-simulation_metric_table:v820',
#     # "traj15": 'run-ujqlvh40-simulation_metric_table:v834',
#     # "traj20": 'run-sk9xeuyz-simulation_metric_table:v835',

#     "groundtruth": 'run-dcclv4id-simulation_metric_table:v538',
#     "flowbot": 'run-79wkxsxs-simulation_metric_table:v710', 
#     "traj1": 'run-m7ohvw2j-simulation_metric_table:v695',
#     "traj5": 'run-zdlpo101-simulation_metric_table:v714',
#     "traj7": 'run-593w003h-simulation_metric_table:v714',
#     "traj10": 'run-w7e49oz6-simulation_metric_table:v708',
#     "traj15": 'run-9tm1ymq0-simulation_metric_table:v707',
#     "traj20": 'run-rmmc64ws-simulation_metric_table:v701',
#     "gt5": 'run-h29bcw85-simulation_metric_table:v557',
#     "gt10": 'run-iodu3rnu-simulation_metric_table:v548',
#     "gt15": 'run-zez98xqr-simulation_metric_table:v537',
#     "gt20": 'run-e3zxo63u-simulation_metric_table:v563',

# }

In [ ]:
artifact_id_dict = {
    # # run-{}-train_metric_table:v0  
    # "flowbot": 'run-pjnc1g2v-simulation_metric_table:v827',  # without mask
    # # "traj1": 'wcx2tyyn',
    # "traj5": 'run-zes1lcl9-simulation_metric_table:v829',
    # "traj10": 'run-bhto6if3-simulation_metric_table:v820',
    # "traj15": 'run-ujqlvh40-simulation_metric_table:v834',
    # "traj20": 'run-sk9xeuyz-simulation_metric_table:v835',

    "groundtruth": 'run-dcclv4id-simulation_metric_table:v538',
    "flowbot1": 'run-y5sluz5a-simulation_metric_table:v835', # without mask
    "flowbot1(with mask)": 'run-m7ohvw2j-simulation_metric_table:v695',
    # "diffuser1": 'run-m7ohvw2j-simulation_metric_table:v695',
}

In [ ]:
import wandb

api = wandb.Api()
for model in artifact_id_dict.keys():
    artifact_id = artifact_id_dict[model]
    artifact = api.artifact(f'r-pad/open_anything_diffusion/{artifact_id}')
    artifact.download()

In [ ]:
import json
import pandas as pd
neat_dfs = pd.DataFrame()
chao_df = pd.DataFrame()
model_order = artifact_id_dict.keys()

object_list = ['Table', 'Window', 'Microwave', 'Phone', 'WashingMachine', 'Kettle', 'Toilet', 'Toy', 'Dishwasher', 'Refrigerator', 'Safe', 'Bucket', 'Door', 'TrashCan', 'Laptop', 'Oven', 'KitchenPot', 'FoldingChair', 'Box', 'StorageFurniture', 'Stapler', 'Switch']
desired_columns_order = ['model'] + object_list

for model in artifact_id_dict.keys():
    artifact_id = artifact_id_dict[model]
    artifact_local_path = f'/home/yishu/open_anything_diffusion/notebooks/artifacts/{artifact_id}/simulation_metric_table.table.json'
    with open(artifact_local_path) as file:
        json_dict = json.load(file)
    df = pd.DataFrame(json_dict["data"], columns=json_dict["columns"]).T
    df.columns = df.loc['index']
    # print(list(df.columns))
    df.drop('index', inplace=True)
    df['model'] = model
    df = df.reindex(columns=desired_columns_order)
    chao_df = pd.concat([chao_df, df])

for metric in ["count", "success_rate", "norm_dist"]:
# for metric in ["success_rate", "norm_dist"]:
    new_df = chao_df.loc[metric].sort_values(by='model', key=lambda x: x.map({v: i for i, v in enumerate(model_order)}))
    neat_dfs = pd.concat([neat_dfs, new_df])

In [ ]:
neat_dfs

In [ ]:
val_objects = ["FoldingChair", "Kettle", "Laptop", "Microwave", "Refrigerator","Stapler", "StorageFurniture", "Switch", "Toilet", "TrashCan",	"Window"]
test_objects = ["Box","Bucket","Dishwasher","Door","KitchenPot", "Oven","Phone","Safe","Table","WashingMachine"]

In [ ]:
import numpy as np
neat_dfs["val_metric"] = np.nan
neat_dfs["test_metric"] = np.nan
reindexed_df = neat_dfs.reset_index()
for model in list(artifact_id_dict.keys()):
    val_count_sum = 0
    test_count_sum = 0
    val_success_rate_sum = 0
    test_success_rate_sum = 0
    val_norm_dist_sum = 0
    test_norm_dist_sum = 0
    for object in object_list:
        count = reindexed_df.query(f"index == 'count' and model=='{model}'")[object].item()
        if object in val_objects:
            val_count_sum += count
            val_success_rate_sum += reindexed_df.query(f"index == 'success_rate' and model=='{model}'")[object].item() * count
            val_norm_dist_sum += reindexed_df.query(f"index == 'norm_dist' and model=='{model}'")[object].item() * count
        else:
            test_count_sum += count
            test_success_rate_sum += reindexed_df.query(f"index == 'success_rate' and model=='{model}'")[object].item() * count
            test_norm_dist_sum += reindexed_df.query(f"index == 'norm_dist' and model=='{model}'")[object].item() * count

    reindexed_df.loc[np.logical_and(reindexed_df["index"]=="count", reindexed_df["model"]==model), "val_metric"] = val_count_sum
    reindexed_df.loc[np.logical_and(reindexed_df["index"]=="count", reindexed_df["model"]==model), "test_metric"] = test_count_sum
    reindexed_df.loc[np.logical_and(reindexed_df["index"]=="success_rate", reindexed_df["model"]==model), "val_metric"] = val_success_rate_sum / val_count_sum
    reindexed_df.loc[np.logical_and(reindexed_df["index"]=="success_rate", reindexed_df["model"]==model), "test_metric"] = test_success_rate_sum / test_count_sum
    reindexed_df.loc[np.logical_and(reindexed_df["index"]=="norm_dist", reindexed_df["model"]==model), "val_metric"] = 1 - val_norm_dist_sum / val_count_sum
    reindexed_df.loc[np.logical_and(reindexed_df["index"]=="norm_dist", reindexed_df["model"]==model), "test_metric"] = 1 - test_norm_dist_sum / test_count_sum

desired_columns_order = ["index", 'model', 'val_metric', 'test_metric'] + object_list

reindexed_df = reindexed_df.reindex(columns=desired_columns_order)

In [ ]:
reindexed_df

In [ ]:
reindexed_df.to_excel('./evaluation_matrix.xlsx')